# APIM ❤️ OpenAI

## Semantic Caching lab
![flow](../../images/semantic-caching.gif)

Playground to try the [semantic caching policy](https://learn.microsoft.com/azure/api-management/azure-openai-semantic-cache-lookup-policy). 

The azure-openai-semantic-cache-lookup policy conducts a cache lookup of responses on Azure OpenAI Chat Completion API and Completion API requests from a pre-configured external cache. It operates by comparing the vector proximity of the prompt to prior requests and using a specific similarity score threshold. Caching responses helps reduce bandwidth and processing demands on the backend Azure OpenAI API, thus reducing latency perceived by API consumers.  
👉🏽 Update: The [Bicep file](main.bicep) was updated to use the new [Azure Managed Redis](https://azure.microsoft.com/en-us/products/managed-redis/) for improved cost efficiency.

[View policy configuration](policy.xml)

### Result
![result](result.png)

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Verify the Azure CLI and the connected Azure subscription](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Make multiple calls using the Azure OpenAI Python SDK](#sdk)
- [🔍 Analyze Semantic Caching performance](#plot)
- [🔍 Show Redis Cache information](#statistics)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

apim_sku = 'Basicv2'

openai_resources = [{"name": "openai1", "location": "uksouth"}]

openai_deployment_name = "gpt-35-turbo"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_model_capacity = 20
openai_api_version = "2024-02-01"

embeddings_deployment_name = "text-embedding-ada-002"
embeddings_model_name = "text-embedding-ada-002"
embeddings_model_version = "2"
rediscache_name = "rediscache"
rediscache_sku = "Balanced_B0" # By default it uses the most cost efficient. Check the docs to choose the right SKU: https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/managed-redis/managed-redis-overview#choosing-the-right-tier

utils.print_ok('Notebook initialized')


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.


In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "apimSku": { "value": apim_sku },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIModelCapacity": { "value": openai_model_capacity },
    "openAIAPIVersion": { "value": openai_api_version },
    "embeddingsModelName": { "value": embeddings_model_name },
    "embeddingsModelVersion": { "value": embeddings_model_version },
    "redisCacheName": { "value": rediscache_name },
    "redisCacheSKU": { "value": rediscache_sku }
  }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    rediscache_host = utils.get_deployment_output(output, 'redisCacheHost', 'Redis Cache Host')
    rediscache_key = utils.get_deployment_output(output, 'redisCacheKey', 'Redis Cache Key (masked)', True)
    rediscache_port = int(utils.get_deployment_output(output, 'redisCachePort', 'Redis Cache Port'))


<a id='sdk'></a>
### 🧪 Make multiple calls using the Azure OpenAI Python SDK

The code below contains a list of questions that will be randomly selected and sent as prompts to the OpenAI API

In [ ]:
from openai import AzureOpenAI
import time, random

runs = 10
questions = ["How to Brew the Perfect Cup of Coffee?", 
             "What are the steps to Craft the Ideal Expresso?", 
             "Tell me how to create the best steaming Java?", 
             "Explain how to make a caffeinated brewed beverage?"]
api_runs = []  # Response Times for each run
for i in range(runs):
    random_question = random.choice(questions)
    messages=[
        {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
        {"role": "user", "content": random_question}
    ]
    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key, api_version=openai_api_version)
    start_time = time.time()
    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    response_time = time.time() - start_time
    print("▶️ Run: ", i+1, f"duration: {response_time:.2f} seconds")
    print("💬 ", random_question)
    api_runs.append(response_time)

<a id='plot'></a>
### 🔍 Analyze Semantic Caching performance

In [ ]:
# plot the results
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 5]
df = pd.DataFrame(api_runs, columns=['Response Time'])
df['Run'] = range(1, len(df) + 1)
df.plot(kind='bar', x='Run', y='Response Time', legend=False)
plt.title('Semantic Caching Performance')
plt.xlabel('Runs')
plt.ylabel('Response Time (s)')
plt.xticks(df['Run'], rotation=0)  # Set x-axis ticks to be the run numbers

average = df['Response Time'].mean()
plt.axhline(y=average, color='r', linestyle='--', label=f'Average: {average:.2f}')
plt.legend()

plt.show()

<a id='statistics'></a>
### 🔍 Show Redis Cache information

In [ ]:

import redis.asyncio as redis
import asyncio

async def get_redis_info():
    r = await redis.from_url(
        f"rediss://:{rediscache_key}@{rediscache_host}:{rediscache_port}"
    )

    info = await r.info()

    print("Redis Server Information:")
    print(f"Used Memory: {info['used_memory_human']}")
    print(f"Cache Hits: {info['keyspace_hits']}")
    print(f"Cache Misses: {info['keyspace_misses']}")
    print(f"Evicted Keys: {info['evicted_keys']}")
    print(f"Expired Keys: {info['expired_keys']}")

    await r.aclose()

await get_redis_info()


### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.